In [2]:

import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

import warnings

warnings.filterwarnings('ignore')

In [3]:
capsule = pd.read_csv('../Documents/GitHub/kaggle-coupon-purchase-prediction/'
                      'input/CAPSULE.csv')
genre = pd.read_csv('../Documents/GitHub/kaggle-coupon-purchase-prediction/'
                      'input/GENRE.csv')
largearea = pd.read_csv('../Documents/GitHub/kaggle-coupon-purchase-prediction/'
                      'input/LARGEAREA.csv')
pref = pd.read_csv('../Documents/GitHub/kaggle-coupon-purchase-prediction/'
                      'input/PREF.csv')
smallarea = pd.read_csv('../Documents/GitHub/kaggle-coupon-purchase-prediction/'
                      'input/SMALLAREA.csv')

### 데이터 불러오기

In [4]:
# train 
area_train = pd.read_csv('coupon_data_project2/coupon_area_train.csv')
detail_train = pd.read_csv('coupon_data_project2/coupon_detail_train.csv', parse_dates=['I_DATE'])
visit_train = pd.read_csv('coupon_data_project2/coupon_visit_train.csv', parse_dates=['I_DATE'])

coupon_list_train = pd.read_csv('coupon_data_project2/coupon_list_train.csv')

# base data
location = pd.read_csv('coupon_data_project2/prefecture_locations.csv')
user_list = pd.read_csv('coupon_data_project2/user_list.csv')

# test data
area_test = pd.read_csv('coupon_data_project2/coupon_area_test.csv')
list_test = pd.read_csv('coupon_data_project2/coupon_list_test.csv')

# submisiion
submission = pd.read_csv('coupon_data_project2/sample_submission.csv')

### 번역하기

- Translate area_train table: SMALL_AREA_NAME

In [5]:
area_train.insert(2, 'en', area_train['SMALL_AREA_NAME'].map(smallarea.set_index('jp')['en']))
area_train.drop(labels=['SMALL_AREA_NAME'], axis = 1, inplace = True)
area_train.rename(columns = {'en':'SMALL_AREA_NAME'}, inplace = True)

- Translate area_train table: SMALL_AREA_NAME

In [6]:
area_train.insert(2, 'en', area_train['PREF_NAME'].map(pref.set_index('jp')['en']))
area_train.drop(labels=['PREF_NAME'], axis = 1, inplace = True)
area_train.rename(columns = {'en':'PREF_NAME'}, inplace=True)

- detail_trian, SMALL_AREA_NAME

In [7]:
detail_train.insert(2, 'en', detail_train['SMALL_AREA_NAME'].map(smallarea.set_index('jp')['en']))
detail_train.drop(labels=['SMALL_AREA_NAME'], axis = 1, inplace=True)
detail_train.rename(columns = {'en':'SMALL_AREA_NAME'}, inplace=True)

- list_copun, large_area_name / small_area_name / capsule_text / genre_name

In [8]:
coupon_list_train.insert(2, 'en', coupon_list_train['large_area_name'].map(largearea.set_index('jp')['en']))
coupon_list_train.drop(labels=['large_area_name'], axis=1, inplace=True)
coupon_list_train.rename(columns = {'en':'large_area_name'}, inplace=True)

In [9]:
coupon_list_train.insert(2, 'en', coupon_list_train['small_area_name'].map(smallarea.set_index('jp')['en']))
coupon_list_train.drop(labels=['small_area_name'], axis=1, inplace=True)
coupon_list_train.rename(columns = {'en':'small_area_name'}, inplace=True)

In [10]:
coupon_list_train.insert(2, 'en', coupon_list_train['CAPSULE_TEXT'].map(capsule.set_index('jp')['en']))
coupon_list_train.drop(labels=['CAPSULE_TEXT'], axis=1, inplace=True)
coupon_list_train.rename(columns = {'en':'CAPSULE_TEXT'}, inplace=True)

In [11]:
coupon_list_train.insert(2, 'en', coupon_list_train['GENRE_NAME'].map(genre.set_index('jp')['en']))
coupon_list_train.drop(labels=['GENRE_NAME'], axis=1, inplace=True)
coupon_list_train.rename(columns = {'en':'GENRE_NAME'}, inplace=True)

- user_list

In [12]:
user_list.insert(2, 'en', user_list['PREF_NAME'].map(pref.set_index('jp')['en']))
user_list.drop(labels=['PREF_NAME'], axis=1, inplace=True)
user_list.rename(columns = {'en':'PREF_NAME'}, inplace=True)

### to_csv

In [21]:
# area_train.to_csv('../Dropbox/data_test/area_train.csv', sep=',', encoding = 'utf-8')
# detail_train.to_csv('../Dropbox/data_test/detail_train.csv', sep=',', encoding = 'utf-8')
# list_train.to_csv('../Dropbox/data_test/list_train.csv', sep=',', encoding = 'utf-8')
# user_list.to_csv('../Dropbox/data_test/user_train.csv', sep=',', encoding = 'utf-8')

## 1) 통합 테이블 형성

### 1-1) VIEW_COUPON_ID_hash -> COUPON_ID_hash 로 변경

In [15]:
# VIEW_COUPON_ID_hash -> COUPON_ID_hash 로 변경
visit_train.rename(columns={'VIEW_COUPON_ID_hash': 'COUPON_ID_hash'}, inplace=True)

### 1-2) detail & vist merge(A column 추가)
- 방문(visit) 과 실제 구매(detail)의 상관관계 분석을 위한 사전 구분자 추가

In [16]:
detail_train['A'] = 'detail'
visit_train['A'] = 'visit'

In [17]:
train = pd.merge(detail_train, visit_train, how='outer')

In [21]:
train.shape[0]

3002176

In [15]:
# train.to_csv('train.csv', sep=',', encoding='utf-8')

In [22]:
def summary_table(table):
    df = pd.DataFrame()
    for i in table.columns:
        name = i
        dtype = table[i].dtype.name
        null = table[i].isnull().sum()
        act = table.shape[0] - null
        distinct = len(table[i].unique())
        data = {'name': name, 'dtype': dtype, 'null': null, 'act':act, 'distinct': distinct}
        df = df.append(data, ignore_index=True)
    
    return df

In [23]:
summary_table(train)

,act,distinct,dtype,name,null
0,168996.0,33.0,float64,ITEM_COUNT,2833180.0
1,3002176.0,2784216.0,datetime64[ns],I_DATE,0.0
2,168996.0,56.0,object,SMALL_AREA_NAME,2833180.0
3,291385.0,168997.0,object,PURCHASEID_hash,2710791.0
4,3002176.0,22833.0,object,USER_ID_hash,0.0
5,3002176.0,32629.0,object,COUPON_ID_hash,0.0
6,3002176.0,2.0,object,A,0.0
7,2833180.0,3.0,float64,PURCHASE_FLG,168996.0
8,2833180.0,467.0,float64,PAGE_SERIAL,168996.0
9,2833180.0,97320.0,object,REFERRER_hash,168996.0


In [25]:
train['Count'] = 1
train.pivot_table(columns=['A'], values= ['Count', 'ITEM_COUNT'], aggfunc=np.sum)

A,detail,visit
Count,168996.0,2833180.0
ITEM_COUNT,241823.0,0.0


### 1-3) location & area_train Merge, 기준: PREF_NAME

In [26]:
area_base = pd.merge(area_train, location, on='PREF_NAME')

In [27]:
print('{}(area_train) + {}(location) = {}(area_base)'.format(area_train.shape, location.shape, area_base.shape))

(138185, 3)(area_train) + (47, 4)(location) = (0, 6)(area_base)


### 1-4) train & area_base Merge, 기준: COUPON_ID_hash

In [28]:
train2 = pd.merge(train, area_base, how='left', on='COUPON_ID_hash')

In [29]:
train2.shape

(3002176, 17)

### 1-5) train2 & user_list Merge, 기준: USER_ID_hash

In [30]:
train3 = pd.merge(train2, user_list, on='USER_ID_hash')

In [31]:
train3.shape

(3002176, 22)

### 1-6) train3 & coupon_list_train Merge, 기준: COUPON_ID_hash

In [33]:
train4 = pd.merge(train3, coupon_list_train, on='COUPON_ID_hash')

In [34]:
train4.shape

(2686202, 45)